<a href="https://colab.research.google.com/github/AAhmadS/NLP_HW2/blob/main/NLP_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/NLP

/content/gdrive/.shortcut-targets-by-id/1Ikn75GANDVFyb0klJe4-EBQJikt4_Ob_/NLP


##Extracting essential data



In [ ]:
!pip install -q hazm
!pip install -q transformers

In [ ]:
!pip install -q dadmatools

In [26]:
from __future__ import unicode_literals
import hazm
from hazm import *

from transformers import pipeline, AutoTokenizer


In [ ]:
import json
with open('./extracted-phrases.json.zip','r',encoding='utf-8') as file:
    data = json.load(file)

##Main code

In [81]:
import itertools

In [29]:
from dadmatools.models.normalizer import Normalizer

dadma_normalizer = Normalizer(
    full_cleaning=True,
    unify_chars=True,
    refine_punc_spacing=True,
    remove_extra_space=True,
    remove_puncs=True,
    remove_html=False,
    remove_stop_word=False,
    replace_email_with="<EMAIL>",
    replace_number_with="<NUM>",
    replace_url_with="",
    replace_mobile_number_with=None,
    replace_emoji_with=None,
    replace_home_number_with=None,
)

dadma_normalizer.normalize("بیا ای عقل کل با من که بردابرد او بینی")


'عقل بردابرد بینی'

In [51]:
corpus = [x[0] for x in hazm.words_list()]

In [102]:
class foreign_word_detector():

  def __init__(self, corpus, lemmatizer, word_tokenizer, sent_tokenizer, normalizer,translator = None):
    self.corpus = [lemmatizer.lemmatize(x) for x in corpus]
    self.lemmatizer = lemmatizer
    self.normalizer = normalizer
    self.translator = translator
    self.word_tokenizer = word_tokenizer
    self.sent_tokenizer = sent_tokenizer

  def detect(self, text):
    sentenced = self.sent_tokenizer.tokenize(text)
    normalized = [self.normalizer.normalize(sent) for sent in sentenced]
    words_list = [self.word_tokenizer.tokenize(sent) for sent in normalized]
    words_list = [self.lemmatizer.lemmatize(x) for x in itertools.chain(*words_list)]

    output = dict()
    out = []
    for word in words_list:
      if word not in self.corpus:
        out.append(word)
        begin = text.find(word)
        output[word] = [begin,begin+len(word)]
    
    return out, output
    

In [105]:
def run():
  detector = foreign_word_detector(corpus, hazm.Lemmatizer(), hazm.WordTokenizer(), hazm.SentenceTokenizer(), dadma_normalizer)

  text = ""
  print("enter your sample text.\nenter an empty string if you want to end the process")
  while True : 
    text = input()
    if text == "": 
      break
    
    out, output = detector.detect(text)
    print(out)
    print(output)


In [106]:
run()

enter your sample text.
enter an empty string if you want to end the process
سیستم کامپیوترم خراب شده.
['سیستم', 'کامپیوترم']
{'سیستم': [0, 5], 'کامپیوترم': [6, 15]}
امروز یک کار خیلی هاردی داشتیم ولی تو کانتریبیوشن خوبی داشتی تنکس.
['کانتریبیوشن', 'تنکس']
{'کانتریبیوشن': [38, 49], 'تنکس': [61, 65]}
تسک امروز هارد بود
['تسک', 'هارد']
{'تسک': [0, 3], 'هارد': [10, 14]}
تسک امروز هاردی بود
['تسک']
{'تسک': [0, 3]}



problem is: using stemmer, we get lots of errors; while using lemmatizer, pronouns are not ommited from the word.


one way to deal with it would be to apply stemming right after dealing with the word origin.

another way would be to find a way to work with the tokenizer introduced in the next cell, since it gives us specific features.

In [108]:
bolbol_tokenizer = AutoTokenizer.from_pretrained('bolbolzaban/gpt2-persian')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [111]:
bolbol_tokenizer.tokenize("سیستم کامپیوترم خراب شده.")

['▁سیستم', '▁کامپیوتر', 'م', '▁خراب', '▁شده', '.']